In [1]:
import pandas as pd
import numpy as np
import pickle
import math
from collections import Counter
from utils_corrected import process_GO_data, evaluate_annotations


In [8]:
def get_predictions_for_fdr_level(predicted_terms_pkl, test_entries, target_fdr):
    """Get predictions at specific FDR level"""
    if target_fdr in predicted_terms_pkl:
        predictions_list = predicted_terms_pkl[target_fdr]
        return {
            test_entries[i]: set(pred_tuple) 
            for i, pred_tuple in enumerate(predictions_list) 
            if pred_tuple  # Only if there are predictions
        }
    else:
        print(f"Warning: FDR {target_fdr} not found in predictions")
        return {}

In [ ]:
ontology = 'BPO'
ontology_type = 'process'

print("=== Loading Test Data ===")
test_embeddings_path = r'processed_data_90_30/{0}_test.npy'.format(ontology_type)
test_tsv_path = r'processed_data_90_30/{0}_test.tsv'.format(ontology_type)

test_embeddings = np.load(test_embeddings_path)
test_GO_df, test_embeddings, test_GO_list, test_GO_annotated = process_GO_data(test_tsv_path, test_embeddings)
test_GT_entries = test_GO_df['Entry'].tolist()

print(f"Loaded {len(test_GT_entries)} test entries")

with open('{0}_mlb.pkl'.format(ontology_type), 'rb') as f:
    mlb = pickle.load(f)

print("=== Loading Predictions ===")
with open('/Users/harsh/Dropbox/Bridge/pfp_final_aug_3/process_test_predictions_and_FDR_new/bpo/predicted_terms_fdr.pkl', 'rb') as f:
    predicted_terms_pkl = pickle.load(f)

print(f"Available FDR levels: {sorted(predicted_terms_pkl.keys())[:10]}...")  # Show first 10
print(f"Number of entries per FDR level: {len(predicted_terms_pkl[0.01])}")

if len(test_GT_entries) == len(predicted_terms_pkl[0.01]):
    print("Entry count matches predictions")
else:
    print(f"Warning: {len(test_GT_entries)} entries vs {len(predicted_terms_pkl[0.01])} predictions")

file_path = '{}_test_disorder_rate.tsv'.format(ontology_type)
data = pd.read_csv(file_path, sep='\t')
data['disorder_rate_bins'] = pd.cut(data['Disorder Rate'], bins=np.arange(0, 1.1, 0.1), right=False)
data_bins_ids_dict = data.groupby('disorder_rate_bins')['Sequence ID'].apply(list).to_dict()

print(f"Disorder rate bins: {list(data_bins_ids_dict.keys())}")
print(f"Entries per bin: {[(str(k), len(v)) for k, v in data_bins_ids_dict.items()]}")

ia_df = pd.read_csv(r'IA_all.tsv', sep='\t', header=None)
ia_df.columns = ['GO', 'IA']
ic_dict = dict(zip(ia_df['GO'], ia_df['IA']))

real_annots_dict = {
    entry: set(go_terms) 
    for entry, go_terms in zip(test_GT_entries, test_GO_list)
}


print("=== Starting Analysis ===")
target_fdr_levels = [0.01, 0.05, 0.10, 0.20]
results_summary = []

for bin_interval, subset_ids in data_bins_ids_dict.items():
    print(f"\nProcessing bin interval: {bin_interval}")
    print(f"  Entries in bin: {len(subset_ids)}")
    
    filtered_real_annots_dict = {
        entry: real_annots_dict[entry]
        for entry in subset_ids 
        if entry in real_annots_dict
    }
    
    print(f"  Entries with ground truth: {len(filtered_real_annots_dict)}")
    
    if not filtered_real_annots_dict:
        print(f"  No ground truth data for interval {bin_interval}, skipping.")
        continue
    
    for target_fdr in target_fdr_levels:
        print(f"    Evaluating FDR {target_fdr:.0%}...")
        
        all_predictions_for_fdr = get_predictions_for_fdr_level(
            predicted_terms_pkl, test_GT_entries, target_fdr
        )
        
        pred_set = {
            entry: all_predictions_for_fdr[entry] 
            for entry in subset_ids 
            if entry in all_predictions_for_fdr
        }
        
        print(f"      Entries with predictions: {len(pred_set)}")
        
        if not pred_set:
            print(f"      No predictions for FDR {target_fdr}, skipping.")
            continue
        
        (f, p, r, s, ru, mi, f_micro, p_micro, r_micro, 
         tp_global, fp_global, fn_global) = evaluate_annotations(
            ic_dict, filtered_real_annots_dict, pred_set
        )
        
        results_summary.append({
            'disorder_interval': f"[{bin_interval.left:.1f}-{bin_interval.right:.1f})",
            'target_FDR': target_fdr,
            'F1': f,
            'Precision': p,
            'Recall': r,
            'Micro_F1': f_micro,
            'S': s,
            'RU': ru,
            'MI': mi,
            'TP': tp_global,
            'FP': fp_global,
            'FN': fn_global,
            'num_entries_gt': len(filtered_real_annots_dict),
            'num_entries_pred': len(pred_set)
        })
        
        print(f"      F1: {f:.3f}, Precision: {p:.3f}, Recall: {r:.3f}, Micro F1: {f_micro:.3f}, Smin {s}")

print("\n=== Analysis Complete ===")
results_df_final = pd.DataFrame(results_summary)

print("\n--- Summary of Disorder Rate Intervals and Target FDR Metrics ---")
print(results_df_final[['disorder_interval', 'target_FDR', 'F1', 'Precision', 'Recall', 'Micro_F1', 'num_entries_gt', 'S']].round(3))

output_file = f'disorder_analysis_results/{ontology_type}_disorder_rate_fdr_analysis.csv'
results_df_final.to_csv(output_file, index=False)


print("\n=== Summary Statistics ===")
print("Performance by Disorder Rate Bin (averaged across FDR levels):")
bin_summary = results_df_final.groupby('disorder_interval')[['F1', 'Precision', 'Recall', 'Micro_F1']].mean()
print(bin_summary.round(3))

print("\nPerformance by FDR Level (averaged across disorder bins):")
fdr_summary = results_df_final.groupby('target_FDR')[['F1', 'Precision', 'Recall', 'Micro_F1']].mean()
print(fdr_summary.round(3))

=== Loading Test Data ===
Loaded 26746 test entries
=== Loading Predictions ===
Available FDR levels: [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]...
Number of entries per FDR level: 26746
Entry count matches predictions
Disorder rate bins: [Interval(0.0, 0.1, closed='left'), Interval(0.1, 0.2, closed='left'), Interval(0.2, 0.3, closed='left'), Interval(0.3, 0.4, closed='left'), Interval(0.4, 0.5, closed='left'), Interval(0.5, 0.6, closed='left'), Interval(0.6, 0.7, closed='left'), Interval(0.7, 0.8, closed='left'), Interval(0.8, 0.9, closed='left'), Interval(0.9, 1.0, closed='left')]
Entries per bin: [('[0.0, 0.1)', 12901), ('[0.1, 0.2)', 4731), ('[0.2, 0.3)', 3083), ('[0.3, 0.4)', 2080), ('[0.4, 0.5)', 1463), ('[0.5, 0.6)', 993), ('[0.6, 0.7)', 569), ('[0.7, 0.8)', 345), ('[0.8, 0.9)', 202), ('[0.9, 1.0)', 178)]
=== Starting Analysis ===

Processing bin interval: [0.0, 0.1)
  Entries in bin: 12901
  Entries with ground truth: 12901
    Evaluating FDR 1%...
      Entrie

In [ ]:
ontology = 'MFO'
ontology_type = 'function'

print("=== Loading Test Data ===")
test_embeddings_path = r'processed_data_90_30/{0}_test.npy'.format(ontology_type)
test_tsv_path = r'processed_data_90_30/{0}_test.tsv'.format(ontology_type)

test_embeddings = np.load(test_embeddings_path)
test_GO_df, test_embeddings, test_GO_list, test_GO_annotated = process_GO_data(test_tsv_path, test_embeddings)
test_GT_entries = test_GO_df['Entry'].tolist()

print(f"Loaded {len(test_GT_entries)} test entries")

with open('{0}_mlb.pkl'.format(ontology_type), 'rb') as f:
    mlb = pickle.load(f)

print("=== Loading Predictions ===")
with open('/Users/harsh/Dropbox/Bridge/pfp_final_aug_3/process_test_predictions_and_FDR_new/bpo/predicted_terms_fdr.pkl', 'rb') as f:
    predicted_terms_pkl = pickle.load(f)

print(f"Available FDR levels: {sorted(predicted_terms_pkl.keys())[:10]}...")  # Show first 10
print(f"Number of entries per FDR level: {len(predicted_terms_pkl[0.01])}")

if len(test_GT_entries) == len(predicted_terms_pkl[0.01]):
    print("Entry count matches predictions")
else:
    print(f"Warning: {len(test_GT_entries)} entries vs {len(predicted_terms_pkl[0.01])} predictions")

file_path = '{}_test_disorder_rate.tsv'.format(ontology_type)
data = pd.read_csv(file_path, sep='\t')
data['disorder_rate_bins'] = pd.cut(data['Disorder Rate'], bins=np.arange(0, 1.1, 0.1), right=False)
data_bins_ids_dict = data.groupby('disorder_rate_bins')['Sequence ID'].apply(list).to_dict()

print(f"Disorder rate bins: {list(data_bins_ids_dict.keys())}")
print(f"Entries per bin: {[(str(k), len(v)) for k, v in data_bins_ids_dict.items()]}")

ia_df = pd.read_csv(r'IA_all.tsv', sep='\t', header=None)
ia_df.columns = ['GO', 'IA']
ic_dict = dict(zip(ia_df['GO'], ia_df['IA']))

real_annots_dict = {
    entry: set(go_terms) 
    for entry, go_terms in zip(test_GT_entries, test_GO_list)
}


print("=== Starting Analysis ===")
target_fdr_levels = [0.01, 0.05, 0.10, 0.20]
results_summary = []

for bin_interval, subset_ids in data_bins_ids_dict.items():
    print(f"\nProcessing bin interval: {bin_interval}")
    print(f"  Entries in bin: {len(subset_ids)}")
    
    filtered_real_annots_dict = {
        entry: real_annots_dict[entry]
        for entry in subset_ids 
        if entry in real_annots_dict
    }
    
    print(f"  Entries with ground truth: {len(filtered_real_annots_dict)}")
    
    if not filtered_real_annots_dict:
        print(f"  No ground truth data for interval {bin_interval}, skipping.")
        continue
    
    for target_fdr in target_fdr_levels:
        print(f"    Evaluating FDR {target_fdr:.0%}...")
        
        all_predictions_for_fdr = get_predictions_for_fdr_level(
            predicted_terms_pkl, test_GT_entries, target_fdr
        )
        
        pred_set = {
            entry: all_predictions_for_fdr[entry] 
            for entry in subset_ids 
            if entry in all_predictions_for_fdr
        }
        
        print(f"      Entries with predictions: {len(pred_set)}")
        
        if not pred_set:
            print(f"      No predictions for FDR {target_fdr}, skipping.")
            continue
        
        (f, p, r, s, ru, mi, f_micro, p_micro, r_micro, 
         tp_global, fp_global, fn_global) = evaluate_annotations(
            ic_dict, filtered_real_annots_dict, pred_set
        )
        
        results_summary.append({
            'disorder_interval': f"[{bin_interval.left:.1f}-{bin_interval.right:.1f})",
            'target_FDR': target_fdr,
            'F1': f,
            'Precision': p,
            'Recall': r,
            'Micro_F1': f_micro,
            'S': s,
            'RU': ru,
            'MI': mi,
            'TP': tp_global,
            'FP': fp_global,
            'FN': fn_global,
            'num_entries_gt': len(filtered_real_annots_dict),
            'num_entries_pred': len(pred_set)
        })
        
        print(f"      F1: {f:.3f}, Precision: {p:.3f}, Recall: {r:.3f}, Micro F1: {f_micro:.3f}, Smin {s}")

print("\n=== Analysis Complete ===")
results_df_final = pd.DataFrame(results_summary)

print("\n--- Summary of Disorder Rate Intervals and Target FDR Metrics ---")
print(results_df_final[['disorder_interval', 'target_FDR', 'F1', 'Precision', 'Recall', 'Micro_F1', 'num_entries_gt', 'S']].round(3))

output_file = f'disorder_analysis_results/{ontology_type}_disorder_rate_fdr_analysis.csv'
results_df_final.to_csv(output_file, index=False)


print("\n=== Summary Statistics ===")
print("Performance by Disorder Rate Bin (averaged across FDR levels):")
bin_summary = results_df_final.groupby('disorder_interval')[['F1', 'Precision', 'Recall', 'Micro_F1']].mean()
print(bin_summary.round(3))

print("\nPerformance by FDR Level (averaged across disorder bins):")
fdr_summary = results_df_final.groupby('target_FDR')[['F1', 'Precision', 'Recall', 'Micro_F1']].mean()
print(fdr_summary.round(3))

=== Loading Test Data ===
Loaded 33343 test entries
=== Loading Predictions ===


/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Available FDR levels: [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]...
Number of entries per FDR level: 33343
Entry count matches predictions
Disorder rate bins: [Interval(0.0, 0.1, closed='left'), Interval(0.1, 0.2, closed='left'), Interval(0.2, 0.3, closed='left'), Interval(0.3, 0.4, closed='left'), Interval(0.4, 0.5, closed='left'), Interval(0.5, 0.6, closed='left'), Interval(0.6, 0.7, closed='left'), Interval(0.7, 0.8, closed='left'), Interval(0.8, 0.9, closed='left'), Interval(0.9, 1.0, closed='left')]
Entries per bin: [('[0.0, 0.1)', 16904), ('[0.1, 0.2)', 6062), ('[0.2, 0.3)', 3768), ('[0.3, 0.4)', 2558), ('[0.4, 0.5)', 1662), ('[0.5, 0.6)', 1037), ('[0.6, 0.7)', 588), ('[0.7, 0.8)', 347), ('[0.8, 0.9)', 170), ('[0.9, 1.0)', 128)]
=== Starting Analysis ===

Processing bin interval: [0.0, 0.1)
  Entries in bin: 16904
  Entries with ground truth: 16904
    Evaluating FDR 1%...
      Entries with predictions: 16904
      F1: 0.843, Precision: 0.989, Recall: 0.735, Mic

In [ ]:
ontology = 'CCO'
ontology_type = 'component'

print("=== Loading Test Data ===")
test_embeddings_path = r'processed_data_90_30/{0}_test.npy'.format(ontology_type)
test_tsv_path = r'processed_data_90_30/{0}_test.tsv'.format(ontology_type)

test_embeddings = np.load(test_embeddings_path)
test_GO_df, test_embeddings, test_GO_list, test_GO_annotated = process_GO_data(test_tsv_path, test_embeddings)
test_GT_entries = test_GO_df['Entry'].tolist()

print(f"Loaded {len(test_GT_entries)} test entries")

with open('{0}_mlb.pkl'.format(ontology_type), 'rb') as f:
    mlb = pickle.load(f)

print("=== Loading Predictions ===")
with open('/Users/harsh/Dropbox/Bridge/pfp_final_aug_3/process_test_predictions_and_FDR_new/bpo/predicted_terms_fdr.pkl', 'rb') as f:
    predicted_terms_pkl = pickle.load(f)

print(f"Available FDR levels: {sorted(predicted_terms_pkl.keys())[:10]}...")  # Show first 10
print(f"Number of entries per FDR level: {len(predicted_terms_pkl[0.01])}")

if len(test_GT_entries) == len(predicted_terms_pkl[0.01]):
    print("Entry count matches predictions")
else:
    print(f"Warning: {len(test_GT_entries)} entries vs {len(predicted_terms_pkl[0.01])} predictions")

file_path = '{}_test_disorder_rate.tsv'.format(ontology_type)
data = pd.read_csv(file_path, sep='\t')
data['disorder_rate_bins'] = pd.cut(data['Disorder Rate'], bins=np.arange(0, 1.1, 0.1), right=False)
data_bins_ids_dict = data.groupby('disorder_rate_bins')['Sequence ID'].apply(list).to_dict()

print(f"Disorder rate bins: {list(data_bins_ids_dict.keys())}")
print(f"Entries per bin: {[(str(k), len(v)) for k, v in data_bins_ids_dict.items()]}")

ia_df = pd.read_csv(r'IA_all.tsv', sep='\t', header=None)
ia_df.columns = ['GO', 'IA']
ic_dict = dict(zip(ia_df['GO'], ia_df['IA']))

real_annots_dict = {
    entry: set(go_terms) 
    for entry, go_terms in zip(test_GT_entries, test_GO_list)
}


print("=== Starting Analysis ===")
target_fdr_levels = [0.01, 0.05, 0.10, 0.20]
results_summary = []

for bin_interval, subset_ids in data_bins_ids_dict.items():
    print(f"\nProcessing bin interval: {bin_interval}")
    print(f"  Entries in bin: {len(subset_ids)}")
    
    filtered_real_annots_dict = {
        entry: real_annots_dict[entry]
        for entry in subset_ids 
        if entry in real_annots_dict
    }
    
    print(f"  Entries with ground truth: {len(filtered_real_annots_dict)}")
    
    if not filtered_real_annots_dict:
        print(f"  No ground truth data for interval {bin_interval}, skipping.")
        continue
    
    for target_fdr in target_fdr_levels:
        print(f"    Evaluating FDR {target_fdr:.0%}...")
        
        all_predictions_for_fdr = get_predictions_for_fdr_level(
            predicted_terms_pkl, test_GT_entries, target_fdr
        )
        
        pred_set = {
            entry: all_predictions_for_fdr[entry] 
            for entry in subset_ids 
            if entry in all_predictions_for_fdr
        }
        
        print(f"      Entries with predictions: {len(pred_set)}")
        
        if not pred_set:
            print(f"      No predictions for FDR {target_fdr}, skipping.")
            continue
        
        (f, p, r, s, ru, mi, f_micro, p_micro, r_micro, 
         tp_global, fp_global, fn_global) = evaluate_annotations(
            ic_dict, filtered_real_annots_dict, pred_set
        )
        
        results_summary.append({
            'disorder_interval': f"[{bin_interval.left:.1f}-{bin_interval.right:.1f})",
            'target_FDR': target_fdr,
            'F1': f,
            'Precision': p,
            'Recall': r,
            'Micro_F1': f_micro,
            'S': s,
            'RU': ru,
            'MI': mi,
            'TP': tp_global,
            'FP': fp_global,
            'FN': fn_global,
            'num_entries_gt': len(filtered_real_annots_dict),
            'num_entries_pred': len(pred_set)
        })
        
        print(f"      F1: {f:.3f}, Precision: {p:.3f}, Recall: {r:.3f}, Micro F1: {f_micro:.3f}, Smin {s}")

print("\n=== Analysis Complete ===")
results_df_final = pd.DataFrame(results_summary)

print("\n--- Summary of Disorder Rate Intervals and Target FDR Metrics ---")
print(results_df_final[['disorder_interval', 'target_FDR', 'F1', 'Precision', 'Recall', 'Micro_F1', 'num_entries_gt', 'S']].round(3))

output_file = f'disorder_analysis_results/{ontology_type}_disorder_rate_fdr_analysis.csv'
results_df_final.to_csv(output_file, index=False)


print("\n=== Summary Statistics ===")
print("Performance by Disorder Rate Bin (averaged across FDR levels):")
bin_summary = results_df_final.groupby('disorder_interval')[['F1', 'Precision', 'Recall', 'Micro_F1']].mean()
print(bin_summary.round(3))

print("\nPerformance by FDR Level (averaged across disorder bins):")
fdr_summary = results_df_final.groupby('target_FDR')[['F1', 'Precision', 'Recall', 'Micro_F1']].mean()
print(fdr_summary.round(3))

=== Loading Test Data ===
Loaded 29679 test entries
=== Loading Predictions ===
Available FDR levels: [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]...
Number of entries per FDR level: 29679
Entry count matches predictions
Disorder rate bins: [Interval(0.0, 0.1, closed='left'), Interval(0.1, 0.2, closed='left'), Interval(0.2, 0.3, closed='left'), Interval(0.3, 0.4, closed='left'), Interval(0.4, 0.5, closed='left'), Interval(0.5, 0.6, closed='left'), Interval(0.6, 0.7, closed='left'), Interval(0.7, 0.8, closed='left'), Interval(0.8, 0.9, closed='left'), Interval(0.9, 1.0, closed='left')]
Entries per bin: [('[0.0, 0.1)', 13038), ('[0.1, 0.2)', 5492), ('[0.2, 0.3)', 3868), ('[0.3, 0.4)', 2646), ('[0.4, 0.5)', 1861), ('[0.5, 0.6)', 1099), ('[0.6, 0.7)', 656), ('[0.7, 0.8)', 407), ('[0.8, 0.9)', 266), ('[0.9, 1.0)', 173)]
=== Starting Analysis ===

Processing bin interval: [0.0, 0.1)
  Entries in bin: 13038
  Entries with ground truth: 13038
    Evaluating FDR 1%...
      Entri

In [3]:
component_df = pd.read_csv("disorder_analysis_results/component_disorder_rate_fdr_analysis.csv")
function_df = pd.read_csv("disorder_analysis_results/function_disorder_rate_fdr_analysis.csv")
process_df = pd.read_csv("disorder_analysis_results/process_disorder_rate_fdr_analysis.csv")

component_fdr10 = component_df[component_df["target_FDR"] == 0.10]
function_fdr10 = function_df[function_df["target_FDR"] == 0.10]
process_fdr10 = process_df[process_df["target_FDR"] == 0.10]

def map_disorder_level(interval):
    low = ["[0.0-0.1)", "[0.1-0.2)", "[0.2-0.3)"]
    med = ["[0.3-0.4)", "[0.4-0.5)", "[0.5-0.6)"]
    high = ["[0.6-0.7)", "[0.7-0.8)", "[0.8-0.9)", "[0.9-1.0]"]
    if interval in low:
        return "Low Disorder (0.0-0.3)"
    elif interval in med:
        return "Medium Disorder (0.3-0.6)"
    elif interval in high:
        return "High Disorder (0.6+)"
    return None

for df in [component_fdr10, function_fdr10, process_fdr10]:
    df["Disorder Level"] = df["disorder_interval"].apply(map_disorder_level)

def summarize_with_mean(df):
    return df.groupby("Disorder Level").agg(
        F_score=("F1", "mean"),
        S=("S", "mean"),
        Precision=("Precision", "mean"),
        Recall=("Recall", "mean"),
        RU=("RU", "mean"),
        MI=("MI", "mean"),
        num_proteins=("num_entries_gt", "mean")
    )

component_summary_mean = summarize_with_mean(component_fdr10)
function_summary_mean = summarize_with_mean(function_fdr10)
process_summary_mean = summarize_with_mean(process_fdr10)

rows_mean = []
for sub, df in {"MF": function_summary_mean, "BP": process_summary_mean, "CC": component_summary_mean}.items():
    for level in ["Low Disorder (0.0-0.3)", "Medium Disorder (0.3-0.6)", "High Disorder (0.6+)"]:
        if level in df.index:
            row = [sub, level] + df.loc[level].tolist()
        else:
            row = [sub, level] + [np.nan] * len(df.columns)
        rows_mean.append(row)

columns_mean = ["Subontology", "Disorder Level", "F_score", "S", "Precision", "Recall", "RU", "MI", "num_proteins"]
final_df_mean = pd.DataFrame(rows_mean, columns=columns_mean)

final_df_mean


/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_16424/2639892136.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Disorder Level"] = df["disorder_interval"].apply(map_disorder_level)
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_16424/2639892136.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Disorder Level"] = df["disorder_interval"].apply(map_disorder_level)
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_16424/2639892136.py:22: SettingWithCopyWarning: 


,Subontology,Disorder Level,F_score,S,Precision,Recall,RU,MI,num_proteins
0,MF,Low Disorder (0.0-0.3),0.900683,2.087337,0.909572,0.891974,1.892139,0.880420,8911.000000
1,MF,Medium Disorder (0.3-0.6),0.896531,2.089621,0.907324,0.886003,1.924947,0.811932,1752.333333
2,MF,High Disorder (0.6+),0.860905,2.456550,0.865926,0.856013,2.225923,1.034620,368.333333
3,BP,Low Disorder (0.0-0.3),0.852927,3.678927,0.916647,0.797554,3.625800,0.621713,6904.666667
4,BP,Medium Disorder (0.3-0.6),0.832046,3.828675,0.897947,0.775185,3.755125,0.744255,1512.000000
5,BP,High Disorder (0.6+),0.803490,5.229658,0.886958,0.734716,5.158668,0.833654,372.000000
6,CC,Low Disorder (0.0-0.3),0.879935,1.146688,0.938958,0.827910,1.132067,0.182080,7465.666667
7,CC,Medium Disorder (0.3-0.6),0.862568,1.383745,0.921894,0.810417,1.362767,0.237254,1868.666667
8,CC,High Disorder (0.6+),0.850177,1.732846,0.921701,0.788959,1.715029,0.234232,443.000000
